In [1]:
# RNN 순환 신경망 (이전 상태를 참고하여 가중치 업데이트)

- RNN(Recurrent Neural Network)

In [2]:
import gdown
file_id = '1MaRptad6gke9_Y_t4b0AGStHMcc_JGfz'
download_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(download_url, 'train_label.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1MaRptad6gke9_Y_t4b0AGStHMcc_JGfz
To: /content/train_label.zip
100%|██████████| 5.47M/5.47M [00:00<00:00, 36.3MB/s]


'train_label.zip'

In [3]:
!unzip train_label.zip

Archive:  train_label.zip
  inflating: train_input.npy         
  inflating: train_label.npy         


In [4]:
import pandas as pd
import numpy as np

In [5]:
train_data = np.load('train_input.npy')
train_label = np.load('train_label.npy')

In [6]:
train_data.shape, train_label.shape

((25000, 174), (25000,))

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(train_data, train_label, random_state=42, train_size=0.2)

In [8]:
# x,y의 데이터를 dictionary 형태로 변환
def mapping_fn(x,y):
  inputs,labels = {'x':x},y
  return inputs, labels

In [9]:
# 데이터를 tensor 형태로 변경하면서 섞고 배치단위의 데이터 셋
import tensorflow as tf
BATCH_SIZE = 16
NUM_EPOCHS = 3
def train_input_fn():
  # x,y 형태의 데이터를 입력 받아서 tensorflow dataset을 만들어줌
  dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  dataset = dataset.shuffle(buffer_size=50000) # 데이터셋 섞기,50000개의 데이터 단위로 섞는다
  dataset = dataset.batch(BATCH_SIZE) # 배치단위로 묶음
  dataset = dataset.repeat(count=NUM_EPOCHS) # 반복
  dataset = dataset.map(mapping_fn) # 배치 단위의 샘플을 mapping_fn 적용해서 input,label형태로 변환
  # 데이터 셋에서 데이터를 하나씩 가져와서 반복하는데 한번 순환하면 초기화 안 함
  iterator = dataset.make_one_shot_iterator()
  return iterator.get_next() # 배치단위로 리턴

def eval_input_fn():
  dataset = tf.data.Dataset.from_tensor_slices((x_val,y_val))
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.map(mapping_fn)
  iterator = dataset.make_one_shot_iterator()
  return iterator.get_next()

In [10]:
# features 자연어를 벡터화한 입력데이터
WORD_EMBEDDING_DIM = 100
HIDDEN_STATE_DIM = 150
DENSE_FEATURE_DIM = 150
learning_rate = 1e-3 # 0.001
def model_fn(features, labels, mode): # mode: train, eval, predict 함수를 만들어 처리
  train = mode == tf.estimator.ModeKeys.TRAIN
  eval = mode == tf.estimator.ModeKeys.EVAL
  predict = mode == tf.estimator.ModeKeys.PREDICT
  # 임베딩 레이어
  embedding_layer = tf.keras.layers.Embedding(input_dim=10000, output_dim=WORD_EMBEDDING_DIM)(features['x'])
  # 과적합 방지를 위해 드랍아웃
  emnbedding_layer = tf.keras.layers.Dropout(rate=0.2)(embedding_layer)
  # 두개의 LSTM 셀을 사용해서 RNN 계층을 생성 각 셀의 크기는 HIDDEN_STATE_DIM으로 설정
  rnn_layers = [tf.nn.rnn_cell.LSTMCell(num_units=size) for size in [HIDDEN_STATE_DIM, HIDDEN_STATE_DIM]]
  # 여러개의 LSTM 셀을 하나의 RNN 계층으로 결합해서 깊이 있는 순환 구조를 생성
  multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
  outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                    inputs= embedding_layer,
                    dtype = tf.float32)
  # 과적합 방지
  outputs = tf.keras.layers.Dropout(0.2)(outputs)
  hidden_layer = tf.keras.layers.Dense(units=DENSE_FEATURE_DIM,
                                       activation=tf.nn.relu)(outputs[:,-1,:]) # FC 완전 연결 부분
  logits = tf.keras.layers.Dropout(0.2)(hidden_layer)
  # 최종 예측값(로짓)
  logits = tf.squeeze(logits, axis=-1) # 차원이 1인 축을 제거해서 단일 값 변환
  sigmoid_logits = tf.nn.sigmoid(logits)
  if predict:
    predictions = {'sentiment':sigmoid_logits}
    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=predictions)
  loss = tf.losses.sigmoid_cross_entropy(labels, logits)
  if eval:
    accuracy = tf.metrics.accuracy(labels-labels, predictions=tf.round(sigmoid_logits))
    eval_metric_ops = {'acc':accuracy}
    return tf.estimator.EstimatorSpec(mode=mode, loss=None,
                                      eval_metric_ops=eval_metric_ops)
  if train:
    global_step = tf.train.get_global_step()
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)
    return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss=loss)

- model
  - tf.estimator.Estimator API를 사용해서 신경망 정의
  - 감성분석처럼 이진분류에 사용 LSTM을 포함해서 순환신경망(RNN)을 기반으로

- LSTM(Long Short Term Memory)
  - RNN의 한 종류, 긴 시퀀스에서 중요한 정보를 잃지 않고 기억할 수 있도록 설계된 메모리 셀 구조
  - 셀 상태와
    - 장기정보
  - 게이트 상태
    - 단기정보
  - 구조
    - 3가지 게이트
      - Forget Gate : 이전 셀의 상태를 보고 어떤 정보를 버릴지 결정
      - Input Gate : 현재 입력 정보를 가지고 새로 저장할 정보
      - Output Gate : 다음 단계로 전달할 정보
    - 동작
      - Forget Gate : 시그모이드 함수 0에 가까우면 정보를 버리고 1에 가까우면 기억
      - Input Gate : sigmoid & tanh 함수조합, 새로운 정보의 비율을 정하고 그 값을 업데이트
      - Output Gate : 다음 레이어 전달할 값을 결정
    - 게이트들의 조합으로 필요한 정보만 남기고 불필요한 정보를 제거

In [11]:
# 영화리뷰
import tensorflow as tf
(x_train,y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)
# 문장의 길이를 맞춰줌
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        # value=0,
                                                        # padding='pre',
                                                        maxlen=200)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                      #  value=0,
                                                      #  padding='pre',
                                                       maxlen=200)
# 모델
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(input_dim=10000, output_dim=100, input_length=200),
  tf.keras.layers.LSTM(units=128, return_sequences=False), # 마지막 타임스템프만 반환
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(x_train, y_train, epochs=3, batch_size=128, validation_split=0.2)
model.evaluate(x_test, y_test)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - accuracy: 0.6035 - loss: 0.6451 - val_accuracy: 0.8486 - val_loss: 0.3672
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.8774 - loss: 0.3026 - val_accuracy: 0.8560 - val_loss: 0.3434
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9202 - loss: 0.2162 - val_accuracy: 0.8430 - val_loss: 0.3640
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8394 - loss: 0.3729


[0.3671480715274811, 0.8424000144004822]

- 벡터화된 데이터를 가지고 LSTM 모델에 적용
- 동일 데이터로 RNN 적용해서 성능 비교

In [12]:
# 하이퍼 파라메터 정의
model_name = 'rnn_classifier_en'
batch_size = 128
num_epochs = 3
vocab_size = 10000
# word_vec_size = 100
# rnn_hidden_size = 128

In [13]:
class Test():
  def __init__(self) -> None:
    pass
  def __call__(self, x):
    print('호출됨 : ',x)
t = Test()
t(10)

# Functional API
Test()(100) # 중간에 추가가 쉬움

호출됨 :  10
호출됨 :  100


In [14]:
# 모델을 클래스
class LSTM_Classifier(tf.keras.Model):
  def __init__(self) -> None:
    super(LSTM_Classifier, self).__init__()
    self.embedding = tf.keras.layers.Embedding(input_dim=10000, output_dim=100, input_length=200)
    self.lstm1 = tf.keras.layers.LSTM(units=128, return_sequences=True) # 모든 은닉레이어의 타임스엡의 출력을 반환
    self.lstm2 = tf.keras.layers.LSTM(units=128) # 스퀀스의 마지막 출력만 반환
    self.dropout = tf.keras.layers.Dropout(rate=0.2)
    self.fc = tf.keras.layers.Dense(units=10, activation='relu')
    self.fc2 = tf.keras.layers.Dense(units=1, activation='sigmoid')
  def call(self, x):
    x = self.embedding(x)
    x = self.lstm1(x)
    x = self.lstm2(x)
    x = self.dropout(x)
    x = self.fc(x)
    x = self.fc2(x)
    return x

In [15]:
model = LSTM_Classifier()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "lstm__classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
# callback early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
cp = tf.keras.callbacks.ModelCheckpoint(filepath=f'{model_name}.keras',monitor ='val_loss', verbose = 1,
                                        save_best_only=True)

In [17]:
history = model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size,
                    validation_data=(x_val, y_val), callbacks=[early_stopping, cp])

Epoch 1/3
194/196 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6458 - loss: 0.5906
Epoch 1: val_loss improved from inf to 2.56033, saving model to rnn_classifier_en.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.6474 - loss: 0.5888 - val_accuracy: 0.5054 - val_loss: 2.5603
Epoch 2/3
195/196 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8832 - loss: 0.2971
Epoch 2: val_loss improved from 2.56033 to 2.32202, saving model to rnn_classifier_en.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.8832 - loss: 0.2970 - val_accuracy: 0.5001 - val_loss: 2.3220
Epoch 3/3
194/196 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9233 - loss: 0.1999
Epoch 3: val_loss improved from 2.32202 to 2.13562, saving model to rnn_classifier_en.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 12s 45ms/step - accuracy: 0.9233 - loss: 0.1999 - val_accuracy: 0.4996 - val_loss: 2.1356


In [21]:
# 모델 로드 -> 예측
best_model = model.load_weights(f'{model_name}.keras')
best_model.evaluate(x_val, y_val)

rnn_classifier_en
